## Extract data from API

In [24]:
import json 
import requests
import pandas as pd

In [26]:
def extract():
    data=requests.get('https://api.open-meteo.com/v1/forecast?latitude=52.52&longitude=13.41&hourly=temperature_2m')
    text=data.text
    dictionary=json.loads(text)
    return dictionary

## transform data into readable dataframe 

In [27]:
def transform(raw_data: dict):
    # Convertit le dictionnaire en un DataFrame en utilisant la fonction json_normalize de pandas
    df=pd.json_normalize(raw_data)
    # Explose les listes présentes dans les colonnes 'hourly.time' et 'hourly.temperature_2m'
    # Cela signifie que si une cellule contient une liste, elle sera "décomposée" en plusieurs lignes
    # Chaque élément de la liste devient une nouvelle ligne dans le DataFrame
    df=df.explode(['hourly.time','hourly.temperature_2m'])

    return df

In [39]:
pip install load_dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## load data

In [40]:
import boto3
import awswrangler as wr
from datetime import datetime
import os
from dotenv import load_dotenv

=> boto3 : est la bibliothèque officielle d'AWS pour interagir avec les services AWS via Python.

=>awswrangler : Simplifie l'accès et la manipulation de données depuis et vers plusieurs sources de données AWS telles que S3, Redshift, Athena, Glue, etc.

=>dotenv est utilisée pour charger les variables d'environnement à partir d'un fichier .env



In [55]:
def load(transformed_data:pd.DataFrame):
    #Chargement des variables d'environnement du fichier .env
    load_dotenv()
    
    #Obtention de la date et de l'heure actuelles
    now=datetime.now()
    year=now.strftime("%Y")
    month = now.strftime("%m")
    day =now.strftime("%d")
    time = now.strftime("%H:%M:%S")

    #crée une session Boto3 en utilisant les clés d'accès AWS et la région spécifiées.
    session=boto3.Session(
        aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
        region_name="us-east-2"
    )

    #On utilise la fonction to_parquet de awswrangler (wr.s3.to_parquet) 
    #pour charger le DataFrame transformed_data dans un bucket S3
    wr.s3.to_parquet(
        df=transformed_data,
        #Le chemin S3 où les données seront chargées est construit en fonction de l'année, du mois, du jour et de l'heure actuels.
        path='s3://etlairflowpipeline/open_meteo' + year + '/'+month +'/' +day +'/'+time + '.parquet',
        boto3_session=session,
    )
        
    

In [56]:
raw_data=extract()
transformed_data=transform(raw_data)
load(transformed_data)

In [1]:
import pandas as pd

# Charger le fichier Parquet dans un DataFrame
df = pd.read_parquet('09_35_15.parquet')

# Afficher les premières lignes du DataFrame
print(df.head())


   latitude  longitude  generationtime_ms  utc_offset_seconds timezone  \
0     52.52  13.419998           0.059962                   0      GMT   
1     52.52  13.419998           0.059962                   0      GMT   
2     52.52  13.419998           0.059962                   0      GMT   
3     52.52  13.419998           0.059962                   0      GMT   
4     52.52  13.419998           0.059962                   0      GMT   

  timezone_abbreviation  elevation hourly_units.time  \
0                   GMT       38.0           iso8601   
1                   GMT       38.0           iso8601   
2                   GMT       38.0           iso8601   
3                   GMT       38.0           iso8601   
4                   GMT       38.0           iso8601   

  hourly_units.temperature_2m       hourly.time  hourly.temperature_2m  
0                          °C  2023-09-30T00:00                   17.2  
1                          °C  2023-09-30T01:00                   15.6  